# URL remediation

## Introduction

- UUID: **94e81d43-0299-45d1-8cb9-5e27b37ff900**
- Started from [issue 16](https://github.com/MISP/misp-playbooks/issues/16)
- State: **Published**
- Purpose : This playbook builds on the **Query URL information** playbook ([Issue #14](https://github.com/MISP/misp-playbooks/issues/14)). 
    - It leverages an existing MISP event containing a URL object to query abuse contact information through **Abuse_Finder**, **LookyLoo**, **FIRST** and **RDAP**. Additionally, it guides you on reporting the URL as a phishing website using platforms such as **Microsoft MSRC**, **Google Safe Browsing**, and **Netcraft**. The playbook also provides detailed instructions for reporting phishing URLs via webforms and email to organisations like **Spamhaus**, **APWG**, **SafeOnWeb**, and others. At the end, a summary is sent to Mattermost.
- Tags: [ "url", "remediation", "phishing" ]
- External resources: **LookyLoo**, **Abusefinder**, **FIRST**, **MMDB**, **Mattermost**, **RDAP**
- Target audience: **SOC, CSIRT, CTI**

![](https://mermaid.ink/img/pako:eNp1VN2OqjAQfpWmiTcGXsALExWPIZFdAms2OWLOFhigR-iQtuweY3z3UyqrMazc0Jn55u-bac80wxzojLqumwjNdQ0zEvhxSNqanVLEI9lFWyKhgZwzzVEkwkInkzMXXM_IOaFFjV9ZxaROqJUr3dRblkKtekXBagWXC7lMJom4Qck2SkwsYj7VpaVkbUV88QlK83LI09t2O98jrktWFaKCa2XrTxDaKOcE-tPeKu3xcHWCAUDW_7RkmSaY_gXzG4LN-5aeIHNsGBcuby3OD4cqDAUGtDV0nGpEa1uh0MZDPSAWaafgT8FFDvIHlB_2oA1gjZlt0mKCwFv2-l9-FL899Yq8Rfg08Zy8bb2fY5ChU5GP-LYQLhrDwUMXEbQorwwHcbTaBzyTqLDQVjw8w27i5X6DWNZAYlYAWUr8UlyUTx1eQGeSFY_J3yEtUDYWkIM67sOKq0ozcXyMs3bNpOo7LG5ZU7FOOWQRvm8c8tqCsK6OreZVmMCHR848KJgoR5twq8-MkZs5oVTXHbvLhxGpPTPWSXVNw-TpqjWUjJXfbY8t9wRj222eI8sgWEPDtAbZoBpIvcv7hH7EfY8CNS_4sH8ayXQa3EDT6UdCD9ShjZEZz83DcO4DJVRX5gnor3NCc0NbV5vLnoiLgbJOY3wSGZ1p2YFDJXZlRWf22ju0a3OmwePMrFxz07ZM_Eb8li__AfbVbqw)

# Playbook

- **URL remediation**
    - Introduction
- **Preparation**
    - PR:1 Initialise environment
    - PR:2 Set helper variables
- **Investigate**
    - IN:1 Choose MISP event
    - IN:2 Abuse contacts via LookyLoo
    - IN:3 Abuse contacts via Abusefinder
    - IN:4 Get geolocation of the IP from MMDB
    - IN:5 Query RDAP
    - IN:6 Contact details via FIRST
- **Containment**
    - CN:1 Report to Microsoft MSRC
    - CN:2 Report to Google Safe Browsing
    - CN:3 Report to Netcraft
    - CN:4 Reporting via web forms
    - CN:5 Reporting via e-mail
    - CN:6 Defang URL
    - CN:7 Export MISP indicators
- **Summary**
    - EN:1 Create the summary of the playbook
    - EN:2 Add the summary as a MISP report
    - EN:3 Send a summary to Mattermost
    - EN:4 Publish MISP event
    - EN:5 End of the playbook
- External references
- Technical details

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. 

The credentials for MISP (**API key**) and other services are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least :

```
misp_url="<MISP URL>"                  # The URL to our MISP server
misp_key="<MISP API KEY>"              # The MISP API key
misp_verifycert=<True or False>        # Indicate if PyMISP should attempt to verify the certificate or ignore errors
mattermost_playbook_user="<MATTERMOST USER>"
mattermost_hook="<MATTERMOST WEBHOOK>"
thehive_url="<THEHIVE URL>"
thehive_key="<THEHIVE API KEY>"
google_sb_api_key="<GOOGLE API KEY>"
```

In [ ]:
# Initialise Python environment
import urllib3
import sys
import json
import base64
import uuid
import time
import re
from prettytable import PrettyTable, MARKDOWN
from IPython.display import Image, display, display_markdown, HTML
from datetime import date
from datetime import datetime
import requests
from pymisp import *
from pymisp.tools import GenericObjectGenerator
import textwrap
import validators
from urllib.parse import urlparse
from pylookyloo import Lookyloo
from pyfaup.faup import Faup
from abuse_finder import domain_abuse, ip_abuse, email_abuse, url_abuse

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
misp_headers = {"Authorization": misp_key,  "Content-Type": "application/json", "Accept": "application/json"}
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n\n".format(misp_url))

## PR:2 Set helper variables

This cell contains **helper variables** that are used in this playbook.

- `playbook_config` : the configuration of the playbook
- `playbook_results` : the results of the playbook

In [ ]:
# Dictionary to playbook results and some of the core objects that are created
playbook_results = {}

playbook_config = {"reporter_email": "info@example.com",
                   "msrc_test_submission": False,
                   "mmdb_url": "https://ip.circl.lu/geolookup/",
                   "skip_tld_search": ["com", "net", "org", "io", "biz", "top"],
                  }

# Investigate

## IN:1 Choose MISP event

In this initial step, the playbook queries a MISP event using its **UUID**. From the selected event, it extracts the **URL object** along with the associated **URL** and **domain** attributes.

Ideally, this event was created using the MISP playbook [Query URL reputation](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_query_url_reputation.ipynb), which helps enrich and validate the URLs.

To proceed, provide the **MISP event UUID** by setting the `event_uuid` variable.

In [ ]:
# Provide the event UUID where you stored the URL object
event_uuid = ""

# Search for the event and the URL
query = False
misp_event = misp.get_event(event_uuid, pythonify=True)
if not "errors" in misp_event:
    print("Found event with UUID \033[92m{}\033[90m".format(event_uuid))
    playbook_results["event"] = misp_event.uuid
    print(" {}".format(misp_event.info))
    for obj in misp_event.objects:
        if obj.name == "url":
            for attribute in obj.attributes:
                if attribute.object_relation == "url":
                    query = attribute.value
                    playbook_results["url"] = query
                if attribute.object_relation == "domain":
                    playbook_results["domain"] = attribute.value
            if query:
                print(" URL: \033[92m{}\033[90m".format(query))
                playbook_results["url_object"] = obj
                print(" URL object UUID: \033[92m{}\033[90m".format(playbook_results["url_object"].uuid))
            else:
                print("Unable to find a \033[91mURL object\033[90m with a URL in the event.")
        if obj.name == "domain-ip":
            for attribute in obj.attributes:
                if attribute.object_relation == "ip":
                    print(" IP: \033[92m{}\033[90m".format(attribute.value))
                    playbook_results["dns"] = {"resolves": attribute.value}
else:
    print("Unable to load event with UUID \033[91m{}\033[90m".format(event_uuid))

## IN:2 Abuse contacts via LookyLoo

The LookyLoo analysis provides us with contact information for the domain, IPs and ASN.  These contacts are also added as MISP attributes.

In [ ]:
# Query Lookyloo
print("Search in event for LookyLoo submit URL")
lookyloo = Lookyloo(lookyloo_url)

def add_takedown_contact(contact, contact_detail):
    attribute = MISPAttribute()
    attribute.value = contact
    attribute.to_ids = False
    attribute.category = "Attribution"
    attribute.type = "email"
    attribute.disable_correlation = False
    attribute.comment = f"Contact for {contact_detail}"
    attribute_misp = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)
    return attribute_misp
                    
                    
lookyloo_uuid = False
for attribute in misp_event.attributes:
    if attribute.type == "link":
        if lookyloo_url in attribute.value and "/tree/" in attribute.value:
            print("Found LookyLoo submit URL {}".format(attribute.value))
            lookyloo_uuid = attribute.value.split("/tree/")[1]
            print(" LookyLoo UUID: \033[92m{}\033[90m".format(lookyloo_uuid))
if lookyloo_uuid:
    playbook_results["takedown"] = { "contacts": [], "IPs": [], "ASNs": [] }
    relationtype = "contact-for"
    takedown_information = lookyloo.get_takedown_information(lookyloo_uuid)
    if takedown_information[0].get("contacts", False):
        for email in takedown_information[0].get("contacts", False):
            if email not in playbook_results["takedown"]["contacts"]:
                attribute_misp = add_takedown_contact(email, "contacts")
                print(" Found \033[92m{}\033[90m".format(email))
                playbook_results["takedown"]["contacts"].append(email)
                if not "errors" in attribute_misp:
                    misp.tag(attribute_misp.uuid, "abuse-contact:lookyloo", True)
                    misp.add_object_reference(playbook_results["url_object"].add_reference(attribute_misp.uuid, relationtype))
    
    if takedown_information[0].get("ips", False):
        for ip in takedown_information[0].get("ips", False):
            for email in takedown_information[0]["ips"][ip]:
                if email not in playbook_results["takedown"]["IPs"]:
                    attribute_misp = add_takedown_contact(email, "IPs")
                    print(" Found \033[92m{}\033[90m".format(email))
                    playbook_results["takedown"]["IPs"].append(email)
                    if not "errors" in attribute_misp:
                        misp.tag(attribute_misp.uuid, "abuse-contact:lookyloo", True)
                        misp.add_object_reference(playbook_results["url_object"].add_reference(attribute_misp.uuid, relationtype))
                        
    if takedown_information[0].get("asns", False):
        for asn in takedown_information[0].get("asns", False):
            for email in takedown_information[0]["asns"][asn]:
                if email not in playbook_results["takedown"]["ASNs"]:
                    attribute_misp = add_takedown_contact(email, "ASNs")
                    print(" Found \033[92m{}\033[90m".format(email))
                    playbook_results["takedown"]["ASNs"].append(email)
                    if not "errors" in attribute_misp:
                        misp.tag(attribute_misp.uuid, "abuse-contact:lookyloo", True)
                        misp.add_object_reference(playbook_results["url_object"].add_reference(attribute_misp.uuid, relationtype))                    
else:
    print("Unable to find a \033[91mLookyLoo submit URL\033[90m in the event.")

### Takedown information enrichment table

The results are now stored in `playbook_results`. Execute the next cell to display them in a table format.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Type", "Enriched"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Enriched"] = "l"
table._max_width = {"Enriched": 50}
for match in playbook_results["takedown"]["contacts"]:
    table.add_row(["LookyLoo", playbook_results["url"], "contacts", match])
for match in playbook_results["takedown"]["IPs"]:
    table.add_row(["LookyLoo", playbook_results["url"], "IPs", match])
for match in playbook_results["takedown"]["ASNs"]:
    table.add_row(["LookyLoo", playbook_results["url"], "ASNs", match])
print(table.get_string(sortby="Type"))
table_lookyloo_info = table

## IN:3 Abuse contacts via abuse_finder

The next cell uses the [abuse_finder](https://github.com/certsocietegenerale/abuse_finder) information to get the most appropriate contact details for abuse reports, both for what concerns the domain and the associated IP. These contacts are added as MISP attributes, either `whois-registrant-name` or `whois-registrant-email`.

In [ ]:
# Query abuse_finder
def process_abuse_details(module_name, item, details, relationtype, comment):
    playbook_results[module_name] = {}
    attribute_mapping = {
        "names": ("Attribution", "whois-registrant-name"),
        "abuse": ("Attribution", "whois-registrant-email")
    }
    email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

    for detail_key, (attribute_category, attribute_type) in attribute_mapping.items():
        if detail_key in details:
            playbook_results[module_name][attribute_type] = []
            for name in details[detail_key]:
                if re.match(email_regex, name):
                    attribute = MISPAttribute()
                    attribute.value = name
                    attribute.to_ids = False
                    attribute.category = attribute_category
                    attribute.type = attribute_type
                    attribute.disable_correlation = False
                    attribute.comment = f"{comment}. Information for {item}"
                    attribute_misp = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)

                    if name not in playbook_results[module_name][attribute_type]:
                        print(" Found \033[92m{}\033[90m".format(name))
                        playbook_results[module_name][attribute_type].append(name)
                        if not "errors" in attribute_misp:
                            misp.tag(attribute_misp.uuid, "abuse-contact:abuse_finder", True) 
                            misp.add_object_reference(playbook_results["url_object"].add_reference(attribute_misp.uuid, relationtype))
                    else:
                        print(attribute_misp)

module_name = "abuse_finder_domain"
module_comment = f"From {module_name}"
relationtype = "contact-for"

print("Start \033[92m{}\033[90m.".format(module_name))
print("Trying \033[92m{}\033[90m".format(playbook_results['domain']))
domain_details = domain_abuse(playbook_results["domain"])
process_abuse_details(module_name, playbook_results["domain"], domain_details, relationtype, module_comment)
print("Finished abuse_finder enrichment for domains.")

module_name = "abuse_finder_ip"
module_comment = f"From {module_name}"
relationtype = "associated-with"

ip = playbook_results.get("dns", {}).get("resolves", [])
if ip:
    print("Trying \033[92m{}\033[90m".format(ip))
    ip_details = ip_abuse(ip)
    process_abuse_details(module_name, ip, ip_details, relationtype, module_comment)
print("Finished abuse_finder enrichment for IPs.\n\n")

### Abuse information enrichment table

The results are now stored in `playbook_results`. Execute the next cell to display them in a table format.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Type", "Enriched"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Enriched"] = "l"
table._max_width = {"Enriched": 50}
for match in playbook_results["abuse_finder_domain"]:
    if len(playbook_results["abuse_finder_domain"][match]) > 0:
        for entry in playbook_results["abuse_finder_domain"][match]:
            table.add_row(["Abuse_finder", playbook_results["url"], match, entry])
for match in playbook_results["abuse_finder_ip"]:
    if len(playbook_results["abuse_finder_ip"][match]) > 0:
        for entry in playbook_results["abuse_finder_ip"][match]:
            table.add_row(["Abuse_finder", playbook_results["url"], match, entry])
print(table.get_string(sortby="Type"))
table_abuse_info = table

## IN:4 Get geolocation of the IP from MMDB

The playbook uses an MMDB service to identify the **geolocation**, with latitude and longitude. In this playbook it uses [ip.circl.lu](https://ip.circl.lu/geolookup/) as MMDB server.

In [ ]:
# Query MMDB
def get_mmdb(value):
    headers = {}
    response = requests.request("GET", "{}{}".format(playbook_config["mmdb_url"], value), headers={})
    if response.status_code == 200:
        result_json = response.json()[0]
        iso_code = result_json.get("country", False).get("iso_code", False)
        return {"iso_code": iso_code}
    else:
        return {}
    
print("Search MMDB")
ip = playbook_results.get("dns", {}).get("resolves", [])
if len(ip) > 0:
    print("Query for \033[92m{}\033[90m".format(ip))
    country = get_mmdb(ip)
    if country:
        playbook_results["geolocation"] = country.get("iso_code", False)
        print(" \033[92m{}\033[90m".format(country))
print("Finished MMDB")

## IN:5 Query RDAP

RDAP or the **Registration Data Access Protocol** is a standardized protocol for accessing registration and ownership details of internet resources like domains, IP addresses, and ASNs, replacing traditional WHOIS. 

This playbook queries the RDAP contact details for the IP associated with the URL. These contacts are added as MISP attributes. In most cases however these contacts will already be added via the previous cells.

In [ ]:
# Query RDAP
rdap_url = "https://rdap.arin.net/registry/ip"
print("Query WHOIS data")
response = requests.get("{}/{}".format(rdap_url, ip))
playbook_results["whois_rdap"] = []
if response.status_code == 200:
    rdap_data = response.json()
    playbook_results["whois_name"] = rdap_data.get("name", False)
    results = []
    entities = rdap_data.get("entities", [])
    for entity in entities:
        handle = entity.get("handle", "N/A")
        roles = entity.get("roles", [])
        vcard = entity.get("vcardArray", [])
        
        email = None
        if len(vcard) == 2 and isinstance(vcard[1], list):
            for entry in vcard[1]:
                if entry[0] == "email":
                    email = entry[3]
                    break
        if email:
            print(" Found \033[92m{}\033[90m - {} {}".format(handle, email, roles))
            results.append({"handle": handle, "roles": roles, "email": email})    
    
    for entity in entities:
        entities_detail = entity.get("entities", [])
        for entity_detail in entities_detail:
            handle = entity_detail.get("handle", "N/A")
            roles = entity_detail.get("roles", [])
            vcard = entity_detail.get("vcardArray", [])
            email = None
            if len(vcard) == 2 and isinstance(vcard[1], list):
                for entry in vcard[1]:
                    if entry[0] == "email":
                        email = entry[3]
                        break
            if email:
                print(" Found \033[92m{}\033[90m - {} {}".format(handle, email, roles))
                results.append({"handle": handle, "roles": roles, "email": email})    
    playbook_results["whois_rdap"] = results
    
    relationtype = "contact-for"
    for entry in playbook_results["whois_rdap"]:
        if entry["email"] and len(entry["email"]) > 0:
            attribute = MISPAttribute()
            attribute.value = entry["email"]
            attribute.to_ids = False
            attribute.category = "Attribution"
            attribute.type = "email"
            attribute.disable_correlation = False
            attribute.comment = "{} - {}".format(entry["handle"], entry["roles"])
            attribute_misp = misp.add_attribute(misp_event.uuid, attribute, pythonify=True)                
            if not "errors" in attribute_misp:
                misp.tag(attribute_misp.uuid, "abuse-contact:rdap", True)
                misp.add_object_reference(playbook_results["url_object"].add_reference(attribute_misp.uuid, relationtype))
else: 
    print("Unable to get a response for \033[91mWHOIS\033[90m data.")
print("Finished query WHOIS")

### RDAP information enrichment table

The results are now stored in `playbook_results`. Execute the next cell to display them in a table format.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Type", "Roles", "Email"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Roles"] = "l"
table.align["Email"] = "l"
for match in playbook_results["whois_rdap"]:
    table.add_row(["WHOIS - RDAP", playbook_results["url"], match["handle"], match["roles"], match["email"]])
print(table.get_string(sortby="Type"))
table_rdap = table

## IN:6 Contact details via FIRST

[FIRST](https://www.first.org/) or the Forum of Incident Response and Security Teams is a global association that brings together incident response and security teams to foster collaboration, information sharing, and best practices for handling cybersecurity incidents. Its members include government agencies, private companies, and academic institutions, all working to improve global cybersecurity resilience. The team list of FIRST is accessible over an API. 

The playbook queries the teams operating in the country where the **TLD** and **IP** is located.

In [ ]:
# Query FIRST
first_url = "https://api.first.org/data/v1"
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

playbook_results["first"] = {}
first_search_country_ip = False
first_search_country_tld = False
print("Search in FIRST.")
if playbook_results.get("geolocation", False):
    first_search_country_ip = playbook_results["geolocation"]
    print(" Search for teams operating in \033[92m{}\033[90m".format(first_search_country_ip))
    first_teams = requests.get("{}/teams?country={}".format(first_url, first_search_country_ip), headers=headers)
    if first_teams.json().get("total", 0) > 0:
        for team in first_teams.json().get("data", []):
            # Skip liaisons (without constituency)
            if len(team.get("constituency", "")) > 0 and len(team.get("email", "")) > 0:
                print("  Found \033[92m{}\033[90m - {}".format(team["team-full"], team.get("constituency", "")))
                print("   {}".format(team.get("email", "")))
                playbook_results["first"][team["team-full"]] = { "country": first_search_country_ip, "constituency": team.get("constituency", ""),
                                                                    "email": team.get("email", "")}

    first_search_country_tld = playbook_results["domain"].split(".")[-1].strip()
    print("\n")
    if first_search_country_tld in playbook_config["skip_tld_search"]:
        print(" Not searching for {}".format(first_search_country_tld))
    else:
        print(" Search for teams operating in \033[92m{}\033[90m".format(first_search_country_tld))
        first_teams = requests.get("{}/teams?country={}".format(first_url, first_search_country_tld), headers=headers)
        if first_teams.json().get("total", 0) > 0:
            for team in first_teams.json().get("data", []):
                # Skip liaisons (without constituency)
                if len(team.get("constituency", "")) > 0 and len(team.get("email", "")) > 0:
                    print("  Found \033[92m{}\033[90m - {}".format(team["team-full"], team.get("constituency", "")))
                    print("   {}".format(team.get("email", "")))
                    playbook_results["first"][team["team-full"]] = { "country": first_search_country_tld,"constituency": team.get("constituency", ""),
                                                                        "email": team.get("email", "")}
else:
    print("Unable to get correct country code to search in FIRST")
print("Finished searching in FIRST.")

### Abuse information enrichment table

The results are now stored in `playbook_results`. Execute the next cell to display them in a table format.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Type", "Enriched", "Country link"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Country link"] = "l"
table.align["Enriched"] = "l"
table._max_width = {"Type": 50}
for match in playbook_results["first"]:
    table.add_row(["FIRST", playbook_results["url"], match, playbook_results["first"][match]["email"], playbook_results["first"][match]["country"]])
print(table.get_string(sortby="Type"))
table_first = table

# Containment

## CN:1 Report to Microsoft MSRC

The next cell allows to report URLs hosted on Microsoft-managed platforms and services, including Microsoft Online Services such as Azure, Bing, OneDrive, and Office 365. It utilises the Microsoft Security Response Center (MSRC) [portal](https://msrc.microsoft.com/report/) for secure and effective reporting.

To prevent accidental submissions, you can set the **testSubmission** (`playbook_config["msrc_test_submission"]`)parameter to True. Change this to False to proceed with an actual report. Additionally, ensure you provide a valid **reporterEmail** (`playbook_config["msrc_reporter_email"]`) for proper attribution and follow-up.

Alternatively, you can also report an unsafe IP or URL to Microsoft Security Intelligence via the form at [https://www.microsoft.com/en-us/wdsi/support/report-unsafe-site-guest](https://www.microsoft.com/en-us/wdsi/support/report-unsafe-site-guest).

In [ ]:
# Report to MSRC
current_datetime = datetime.now()
payload = {
    "date": current_datetime.strftime("%Y-%m-%d"),
    "time": current_datetime.strftime("%H:%M:%S"),
    "timeZone": time.strftime("%z"),
    "threatType": "URL",
    "incidentType": "Phishing",
    "sourceUrl": playbook_results["url"],
    "testSubmission": playbook_config["msrc_test_submission"],
    "reporterName": "MISP playbook",
    "reporterEmail": playbook_config["reporter_email"],
    "reportNotes": "MISP playbook"
}

print("Start submitting to MSRC")
msrc_url = "https://api.msrc.microsoft.com/report/v3.0/abuse"
headers = {
  "Content-Type": "application/json"
}
playbook_results["msrc"] = False
response = requests.post(msrc_url, json=payload, headers=headers)
if response.json().get("code", "") == "OK":
    print(" Submitted \033[92m{}\033[90m".format(response.text))
    playbook_results["msrc"] = True
else:
    print(" Failed submitting \033[91m{}\033[90m".format(response.text))
print("End submitting to MSRC")

## CN:2 Report to Google Safe Browsing

You can submit the URL to [Google Safe Browsing](https://safebrowsing.google.com/safebrowsing/report_general/). Note that you need access to the API and a valid API key (defined in `google_sb_api_key`).

In [ ]:
# Submit to Google Safe Browsing
google_sb_url = "https://safebrowsing.googleapis.com/v4/threatMatches:find?key="
payload = {
    "client": {
        "clientId": "MISP playbook",
        "clientVersion": "1.0"
    },
    "threatInfo": {
        "threatTypes": ["SOCIAL_ENGINEERING"], 
        "platformTypes": ["ANY_PLATFORM"],
        "threatEntryTypes": ["URL"],
        "threatEntries": [{"url": playbook_results["url"]}]
    }
}
print("Start submitting to Google Safe Browsing")
playbook_results["google_sb"] = False
response = requests.post("{}{}".format(google_sb_url, google_sb_api_key), json=payload)
if response.status_code == 200:
    print(" URL \033[92msubmitted successfully\033[90m {}".format(response.text))
    playbook_results["google_sb"] = True
else:
    print(" Failed submitting {} - \033[91m{}\033[90m".format(response.status_code, response.text))
print("End submitting to Google Safe Browsing")

## CN:3 Report to Netcraft

Similar as the previous cells, you can also report the URL to [https://www.netcraft.com/](https://www.netcraft.com/).

In [ ]:
# Submit to Netcraft
netcraft_url = "https://report.netcraft.com/api/v3/report/urls"
payload = {
    "email": playbook_config["reporter_email"],
    "urls": [{"reason":"phishing", "url": playbook_results["url"]}]
}
print("Start submitting to Netcraft")
playbook_results["netcraft"] = False
response = requests.post(netcraft_url, json=payload)
if response.status_code == 200:
    print(" URL \033[92msubmitted successfully\033[90m {}".format(response.text))
    playbook_results["netcraft"] = True
else:
    print(" Failed submitting {} - \033[91m{}\033[90m".format(response.status_code, response.text))
print("End submitting to Netcraft")

## CN:4 Reporting via web forms

Not all services allow you to report phishing URLs, or suspicious URLs, via an API. Some services require to report the URL via a web form.

### PhishTank

You can submit a URL to PhishTank via [https://phishtank.org/add_web_phish.php](https://phishtank.org/add_web_phish.php). Note that you need to be **logged in** to PhishTank.

## CN:5 Reporting via e-mail

An alternative way of reporting suspicious URLs is via e-mail. Take great care in **defanging** the URL (the next cell can help you with that).

### Spamhaus

Report phishing via **phish@spamhaus.org**

### APWG (Anti-Phishing Working Group)

Report phishing via **reportphishing@apwg.org**

### OpenPhish

Report phishing via **report@openphish.com**

### SafeOnWeb

Report phishing via **verdacht@safeonweb.be**
 

## CN:6 Defang URL

A cell to simplify the defanging of a URL.

In [ ]:
# Defang the URL
defang_url = playbook_results["url"].replace("http://", "hxxp://").replace("https://", "hxxps://").replace(".", "[.]")
print("Defanged URL: {}".format(defang_url))

## CN:7 Export MISP indicators

The next section first **queries MISP for the indicators added to the event** linked to the execution of this playbook. Only those indicators with to_ids set to True are returned. You can use this list, or the **raw** list as input for your web filtering devices.

In [ ]:
# Get all the indicators for our event and store this is in a table. We can also use this for the summary.
indicator_search = misp.search("attributes", uuid=misp_event.uuid, to_ids=True, pythonify=True)
indicator_raw_list = []
indicator_table = PrettyTable()
if len(indicator_search) > 0:
    indicator_table.field_names = ["Type", "Category", "Indicator", "Comment"]
    indicator_table.align["Type"] = "l"
    indicator_table.align["Category"] = "l"
    indicator_table.align["Indicator"] = "l"
    indicator_table.align["Comment"] = "l"
    indicator_table.border = True
    for indicator in indicator_search:
        if indicator.value not in indicator_raw_list:
            indicator_table.add_row([indicator.type, indicator.category, indicator.value, indicator.comment])
            indicator_raw_list.append(indicator.value)
    print("Got \033[92m{}\033[90m indicator(s) from the event \033[92m{}\033[90m ({}).\n\n".format(len(indicator_raw_list), misp_event.info, misp_event.id))
else:
    print("\033[93mNo indicators found in the event \033[92m{}\033[90m ({})".format(misp_event.info, misp_event.id))

### Raw list of MISP indicators

The indicators are now stored in `indicator_search` (as Python objects) and `indicator_raw_list` (in raw format, only the indicators). Execute the next cell to display them in a table format. The table is also included in the summary sent to Mattermost and TheHive.

In [ ]:
if len(indicator_raw_list) > 0:
    print(indicator_table.get_string(sortby="Type"))
    print("\n\nIndicator list in raw format:")
    print("---------------------------------------------------")
    for el in indicator_raw_list:
        print("{}".format(el))
    print("---------------------------------------------------")

# Closure

In this **closure** or end step we create a **summary** of the actions that were performed by the playbook. The summary is printed in the playbook and can also be send to a chat channel. 

## EN:1 Create the summary of the playbook

The next section creates a summary and stores the output in the variable `summary` in Markdown format.

In [ ]:
summary = "# MISP Playbook summary\nURL remediation.\n\n"

current_date = datetime.now()
formatted_date = current_date.strftime("%Y-%m-%d")
summary += "## Overview\n\n"
summary += "- Date: **{}**\n".format(formatted_date)
summary += "- URL: **{}**\n".format(playbook_results["url"])
summary += "- Domain: **{}**\n".format(playbook_results["domain"])
summary += "- Resolves to: **{}**\n".format(playbook_results["dns"]["resolves"])

summary += "\n\n"
summary += "## Abuse contacts\n\n"
for match in playbook_results["takedown"]["contacts"]:
    summary += "- Contacts: **{}** (LookyLoo)\n".format(match)
for match in playbook_results["abuse_finder_domain"]:
    if len(playbook_results["abuse_finder_domain"][match]) > 0:
        for entry in playbook_results["abuse_finder_domain"][match]:
            summary += "- Contacts: **{}**\n (Abusefinder)\n".format(entry)
for match in playbook_results["takedown"]["IPs"]:
    summary += "- Contacts - IP: **{}** (LookyLoo)\n".format(match)
for match in playbook_results["abuse_finder_ip"]:
    if len(playbook_results["abuse_finder_ip"][match]) > 0:
        for entry in playbook_results["abuse_finder_ip"][match]:
            summary += "- Contacts - IP: **{}**\n (Abusefinder)\n".format(entry)
for match in playbook_results["takedown"]["ASNs"]:
    summary += "- Contacts - ASN: **{}** (LookyLoo)\n".format(match)
for match in playbook_results["whois_rdap"]:
    if not match["email"] == "None":
        summary += "- Contacts - IP: **{}** as {} for {} (RDAP)\n".format(match["email"], match["roles"],match["handle"])
summary += "\n\n"


summary += "## FIRST teams\n"
summary += "Overview of FIRST teams. First column indicates the country they are operating in.\n"
for match in playbook_results["first"]:
    if len(playbook_results["first"][match]["email"].strip()) > 0:
        summary += "- ({}): {} **{}** ({})\n".format(playbook_results["first"][match]["country"], match, playbook_results["first"][match]["email"], playbook_results["first"][match]["constituency"])
summary += "\n\n"


summary += "## URL reports\n"
if playbook_results["msrc"]:
    summary += "- Reported to **Microsoft** MSRC\n"
if playbook_results["google_sb"]:
    summary += "- Reported to **Google** Safe Browsing\n"
if playbook_results["netcraft"]:
    summary += "- Reported to **Netcraft**\n"

summary += "## Indicators\n\n"
summary += "### Indicators table\n\n"
if len(indicator_raw_list) > 0:
    indicator_table.set_style(MARKDOWN)
    summary += indicator_table.get_string(sortby="Type")
    summary += "\n\n\n" 
    summary += "### Indicators in **raw format**\n\n"
    for indicator in indicator_raw_list:
        summary += "{}\n\n".format(indicator)
    summary += "\n" 
else:
    summary += "There are no indicators"
summary += "\n\n"

summary += "## LookyLoo\n\n"
summary += "### Overview table\n\n"
table_lookyloo_info.set_style(MARKDOWN)
summary += table_lookyloo_info.get_string()
summary += "\n\n"

summary += "## Abusefinder\n\n"
summary += "### Overview table\n\n"
table_abuse_info.set_style(MARKDOWN)
summary += table_abuse_info.get_string()
summary += "\n\n"

summary += "## RDAP\n\n"
summary += "### Overview table\n\n"
table_rdap.set_style(MARKDOWN)
summary += table_rdap.get_string()
summary += "\n\n"

summary += "## FIRST\n\n"
summary += "### Overview table\n\n"
table_first.set_style(MARKDOWN)
summary += table_first.get_string()
summary += "\n\n"
                                                                            
print("The \033[92msummary\033[90m of the playbook is available.\n\n")

#### Print the summary

In [ ]:
display_markdown(summary, raw=True)

## EN:2 Add the summary as a MISP report

Add the summary as a MISP report to the event.

In [ ]:
event_title = "URL remediation"
print("Creating MISP report \033[92m{}\033[90m".format(event_title))
chunk_size = 61500
for i in range(0, len(summary), chunk_size):
    chunk = summary[i:i + chunk_size]
    event_report = MISPEventReport()
    event_title_edit = event_title
    if i > 0:
        event_title_edit = "{} ({} > {})".format(event_title, i, i + chunk_size)
    event_report.name = event_title_edit
    event_report.content = chunk
    result = misp.add_event_report(misp_event.id, event_report)
    if "EventReport" in result:
        print(" Report ID: \033[92m{}\033[90m".format(result.get("EventReport", []).get("id", 0)))
    else:
        print("Failed to create report for \033[91m{}\033[90m.".format(event_title))
print("Finished creating report")

## EN:3 Send a summary to Mattermost

Now you can send the summary to Mattermost. You can send the summary in two ways by selecting one of the options for the variable `send_to_mattermost_option` in the next cell.

- The default option where the entire summary is in the **chat**, or
- a short intro and the summary in a **card**

For this playbook we rely on a webhook in Mattermost. You can add a webhook by choosing the gear icon in Mattermost, then choose Integrations and then **Incoming Webhooks**. Set a channel for the webhook and lock the webhook to this channel with *"Lock to this channel"*.

In [ ]:
send_to_mattermost_option = "via a chat message"
#send_to_mattermost_option = "via a chat message with card"

In [ ]:
message = False

if send_to_mattermost_option == "via a chat message":
    message = {"username": mattermost_playbook_user, "text": summary}
elif send_to_mattermost_option == "via a chat message with card":
    message = {"username": mattermost_playbook_user, "text": intro, "props": {"card": summary}}

if message:
    r = requests.post(mattermost_hook, data=json.dumps(message))
    r.raise_for_status()
if message and r.status_code == 200:
    print("Summary is \033[92msent to Mattermost.\n")
else:
    print("\033[91mFailed to sent summary\033[90m to Mattermost.\n")

## EN:4 Publish MISP event 

As a final step, you can choose the **publish** the MISP event. 

Publishing MISP events makes the event available to your users and, depending on the synchronisation and distribution rules, will also sync it with other connected MISP servers. Publishing an event also typically makes the indicators available for your security controls to import them in their ruleset.

In [ ]:
misp.publish(misp_event.uuid)
misp.untag(misp_event.uuid, "workflow:state=\"incomplete\"")
misp.tag(misp_event.uuid, "workflow:state=\"complete\"", True)
print("Event {} ({} - {}) is \033[92mpublished.\n".format(misp_event.info, misp_event.id, misp_event.uuid))

## EN:5 End of the playbook 

In [ ]:
print("\033[92m End of the playbook")

## External references

- [The MISP Project](https://www.misp-project.org/)
- [Mattermost](https://mattermost.com/)


## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
PrettyTable
ipywidgets
pylookyloo
pyfaup
```

These **MISP modules** need to be enabled
- urlscan
- dns

### Colour codes

The output from Python displays some text in different colours. These are the colour codes

```
Red = '\033[91m'
Green = '\033[92m'
Blue = '\033[94m'
Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'
Black = '\033[90m'
Default = '\033[99m'
```